In [4]:
pip install py2neo

     |████████████████████████████████| 177 kB 5.2 MB/s 


In [5]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship

In [6]:
uri = 'bolt://35.175.116.172:7687'
pwd = 'alternation-pyramid-semiconductor'
user = 'neo4j'

In [7]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [8]:
conn = Neo4jConnection(uri=uri, user= user, pwd=pwd)


In [9]:
result = conn.query('MATCH (n) RETURN COUNT(n) AS ct')
print(result)

[<Record ct=10>]


In [12]:
visual = conn.query('MATCH (n) RETURN n')
visual

[<Record n=<Node id=0 labels=frozenset({'SOURCE'}) properties={'name': 'June year  forecast'}>>,
 <Record n=<Node id=1 labels=frozenset({'SOURCE'}) properties={'name': 'two Paul Keating'}>>,
 <Record n=<Node id=2 labels=frozenset({'SOURCE'}) properties={'name': 'annual  Keating'}>>,
 <Record n=<Node id=3 labels=frozenset({'SOURCE'}) properties={'name': 'kind World War Keating'}>>,
 <Record n=<Node id=4 labels=frozenset({'SOURCE'}) properties={'name': 'Australia'}>>,
 <Record n=<Node id=5 labels=frozenset({'TARGET'}) properties={'name': '87  budget'}>>,
 <Record n=<Node id=6 labels=frozenset({'TARGET'}) properties={'name': 'two  Parliament'}>>,
 <Record n=<Node id=7 labels=frozenset({'TARGET'}) properties={'name': 'figures'}>>,
 <Record n=<Node id=8 labels=frozenset({'TARGET'}) properties={'name': 'Australia trading holocaust'}>>,
 <Record n=<Node id=9 labels=frozenset({'TARGET'}) properties={'name': 'recession'}>>]

In [13]:
allRelations = conn.query('MATCH ()-->() RETURN count(*);')     
allRelations


[<Record count(*)=5>]